In [6]:
import pandas as pd
import folium
import numpy as np
from scipy.spatial import distance

# 1. Load Data
df = pd.read_csv('../data/clustered_crime_data.csv')

# 2. Extract the "Heart" of Each Cluster
# We find the mathematical center (mean Lat/Lon) of each danger zone.
# This tells us exactly where to park a patrol car for maximum effect.
patrol_points = df.groupby('Cluster')[['Latitude', 'Longitude']].mean().reset_index()
print("🎯 Identified Patrol Targets:")
print(patrol_points)

# 3. Solve the Route (The "Nearest Neighbor" Algorithm)
# We start at the first cluster and always go to the closest unvisited one.
coordinates = patrol_points[['Latitude', 'Longitude']].values
route_indices = [0]  # Start at Cluster 0
current_idx = 0

while len(route_indices) < len(coordinates):
    # Calculate distances from current point to all others
    dists = distance.cdist([coordinates[current_idx]], coordinates, metric='euclidean')[0]
    
    # Set visited distances to infinity so we don't go back
    dists[route_indices] = np.inf
    
    # Find nearest unvisited point
    nearest_idx = np.argmin(dists)
    route_indices.append(nearest_idx)
    current_idx = nearest_idx

# Add start point to end to close the loop (Return to Base)
route_indices.append(route_indices[0])
optimized_route = coordinates[route_indices].tolist()

# 4. Visualize the "Sentinel Path"
# Create a dark map
base_lat = patrol_points['Latitude'].mean()
base_long = patrol_points['Longitude'].mean()
sentinel_map = folium.Map(location=[base_lat, base_long], zoom_start=11, tiles='cartodbdark_matter')

# Add Markers for the Checkpoints
for i, row in patrol_points.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"🛑 Patrol Checkpoint: Zone {int(row['Cluster'])}",
        icon=folium.Icon(color='red', icon='shield', prefix='fa')
    ).add_to(sentinel_map)

# Draw the Optimal Flight Path (Glowing Cyan Line)
folium.PolyLine(
    optimized_route,
    color='#00FFFF', # Cyan ("Cyberpunk")
    weight=5,
    opacity=0.8,
    dash_array='10'  # Dashed line for "Future" look
).add_to(sentinel_map)

# 5. Save
sentinel_map.save('../sentinel_patrol_route.html')
print("✅ OPTIMAL ROUTE CALCULATED.")
print("Open 'sentinel_patrol_route.html' to view the flight path.")

🎯 Identified Patrol Targets:
   Cluster   Latitude  Longitude
0        0  41.768784 -87.680448
1        1  41.960707 -87.677648
2        2  41.875726 -87.643086
3        3  41.901264 -87.747718
4        4  41.747624 -87.599332
✅ OPTIMAL ROUTE CALCULATED.
Open 'sentinel_patrol_route.html' to view the flight path.
